# 集計表の作成

In [1]:
import pandas as pd
import numpy as np
from glob import glob

## データの抽出と結合

In [2]:
filepaths = glob('source/請求書*.xlsx')
filepaths

['source/請求書_10月_TTTInc.xlsx',
 'source/請求書_11月_ICHIGO.xlsx',
 'source/請求書_11月_XYZ株式会社.xlsx',
 'source/請求書_12月_CDF商事.xlsx',
 'source/請求書_12月_STR_Corp.xlsx',
 'source/請求書_12月_TTTInc.xlsx',
 'source/請求書_12月_株式会社ABC.xlsx']

In [3]:
# エクセルファイルから必要な情報を抽出する関数
def extract(filepath):
    _df = pd.read_excel(filepath)
    columns = _df.iloc[10, [1, 2, 4, 10, 11, 14]]
    df = _df.iloc[11:23, [1, 2, 4, 10, 11, 14]]
    df.columns = columns
    df['企業名'] = _df.iloc[2, 0]
    df['企業コード'] = _df.iloc[3, 4]
    df['請求書No'] = _df.iloc[2, 12]
    df['発行日'] = _df.iloc[3, 12]
    df['発行者'] = _df.iloc[4, 12]
    df['発行者コード'] = _df.iloc[4, 13]
    return df

In [4]:
# データの結合
df = pd.DataFrame()
for filepath in filepaths:
    _df = extract(filepath)
    df = pd.concat([df, _df])

# 欠損値削除、indexのリセット、カラム順の調整
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df = df.iloc[:, [6,7,8,9,10,0,1,2,3,4,5]]

In [5]:
df

10,企業名,企業コード,請求書No,発行日,発行者,日付,商品コード,品名,数量[L] or [-],単価[¥/L] or [¥],金額
0,TTT Inc.,6,012,2020/10/31,岸辺,2020-10-04 00:00:00,GSL0001,レギュラー,20,120,2400
1,TTT Inc.,6,012,2020/10/31,岸辺,2020-10-05 00:00:00,GSL0002,ハイオク,45,160,7200
2,TTT Inc.,6,012,2020/10/31,岸辺,2020-10-06 00:00:00,GSL0003,軽油,50,100,5000
3,TTT Inc.,6,012,2020/10/31,岸辺,2020-10-07 00:00:00,GDS0002,コロン,3,800,2400
4,TTT Inc.,6,012,2020/10/31,岸辺,2020-10-08 00:00:00,CMT0004,オイル交換,1,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...
67,株式会社ABC,1,003,2020/12/28,桜田,2020-12-20 00:00:00,CMT0004,オイル交換,1,1000,1000
68,株式会社ABC,1,003,2020/12/28,桜田,2020-12-22 00:00:00,CMT0005,タイヤ交換,1,4000,4000
69,株式会社ABC,1,003,2020/12/28,桜田,2020-12-22 00:00:00,GDS0001,タイヤ,4,8000,32000
70,株式会社ABC,1,003,2020/12/28,桜田,2020-12-22 00:00:00,GDS0002,コロン,1,800,800


In [6]:
df.to_excel('output/all_data_02.xlsx', index=True)

## 集計データの出力

In [7]:
menbers = df['発行者'].unique()
menber = menbers[0]
menber

'岸辺'

In [8]:
# まずは岸辺さんのみのい集計表を作る
_df = df[df['発行者'] == menber]

# 岸辺さんの合計金額
_df['金額'].sum()

120300

In [9]:
companies = _df['企業名'].unique()
companies

array(['TTT Inc.', 'ICHIGO'], dtype=object)

In [10]:
_df[_df['企業名'] == companies[0]]['金額'].sum()

89400

In [11]:
_df[_df['企業名'] == companies[1]]['金額'].sum()

30900

In [12]:
total_earnings = _df['金額'].sum()
pd.DataFrame({'担当者': menber, '企業名' : '全体', '金額' : total_earnings}, index=[0])

,担当者,企業名,金額
0,岸辺,全体,120300


In [13]:
earnings = _df[_df['企業名'] == companies[0]]['金額'].sum()
pd.DataFrame({'担当者': menber, '企業名' : companies[0], '金額' : earnings}, index=[1])

,担当者,企業名,金額
1,岸辺,TTT Inc.,89400


In [14]:
# 辞書はdictクラスでも書ける
{'担当者': menber, '企業名' : companies[0], '金額' : earnings}

{'担当者': '岸辺', '企業名': 'TTT Inc.', '金額': 89400}

In [15]:
dict(担当者=menber, 企業名=companies[0], 金額=earnings)

{'担当者': '岸辺', '企業名': 'TTT Inc.', '金額': 89400}

## for文でまとめていく
- for文を書くコツは最初から繰り返し処理を書こうとするのではなく、最初は1つの処理で実行し、その処理を羅列することから始める
- 上記のようなコードを先に書いて、羅列すると、for文が書きやすくなる

In [16]:
members = df['発行者'].unique()
result = []

for member in members:
    _df = df[df['発行者'] == member]
    total_earnings = _df['金額'].sum()
    result.append(dict(担当者=member, 企業名='全体', 金額=total_earnings))
    
    companies = _df['企業名'].unique()
    for company in companies:
        earnings = _df[_df['企業名'] == company]['金額'].sum()
        result.append(dict(企業名=company, 金額=earnings))

In [17]:
pd.DataFrame(result, columns=['担当者', '企業名', '金額'])

,担当者,企業名,金額
0,岸辺,全体,120300
1,NaN,TTT Inc.,89400
2,NaN,ICHIGO,30900
3,山岡,全体,35800
4,NaN,XYZ株式会社,35800
5,桜田,全体,154400
6,NaN,CDF商事,39000
7,NaN,STR Corp.,55300
8,NaN,株式会社ABC,60100


# VLOOKUP関数
- pd.mergeを使う

In [18]:
df_master = pd.read_excel('source/商品情報マスタ.xlsx')
df_order = pd.read_excel('source/注文表.xlsx')

In [19]:
df_order

,日付,商品コード,数量
0,2020-12-01,GSL0001,40
1,2020-12-02,GSL0002,50
2,2020-12-03,GSL0001,35
3,2020-12-15,GSL0004,60
4,2020-12-15,CMT0003,2
5,2020-12-18,GSL0004,15
6,2020-12-21,CMT0003,4
7,2020-12-22,GSL0004,22
8,2020-12-22,CMT0005,3
9,2020-12-25,GSL0004,32


In [20]:
df_master

,商品コード,品名,単価[¥/L] or [¥]
0,GSL0001,レギュラー,120
1,GSL0002,ハイオク,160
2,GSL0003,軽油,100
3,GSL0004,灯油,80
4,CMT0001,洗車 A,500
5,CMT0002,洗車 B,1500
6,CMT0003,洗車 C,3000
7,CMT0004,オイル交換,1000
8,CMT0005,タイヤ交換,4000
9,GDS0001,タイヤ,8000


In [21]:
df_merged = pd.merge(df_order, df_master, how='left', on='商品コード')
df_merged

,日付,商品コード,数量,品名,単価[¥/L] or [¥]
0,2020-12-01,GSL0001,40,レギュラー,120
1,2020-12-02,GSL0002,50,ハイオク,160
2,2020-12-03,GSL0001,35,レギュラー,120
3,2020-12-15,GSL0004,60,灯油,80
4,2020-12-15,CMT0003,2,洗車 C,3000
5,2020-12-18,GSL0004,15,灯油,80
6,2020-12-21,CMT0003,4,洗車 C,3000
7,2020-12-22,GSL0004,22,灯油,80
8,2020-12-22,CMT0005,3,タイヤ交換,4000
9,2020-12-25,GSL0004,32,灯油,80


In [22]:
df_merged['合計金額'] = df_merged['数量'] * df_merged['単価[¥/L] or [¥]']
df_merged

,日付,商品コード,数量,品名,単価[¥/L] or [¥],合計金額
0,2020-12-01,GSL0001,40,レギュラー,120,4800
1,2020-12-02,GSL0002,50,ハイオク,160,8000
2,2020-12-03,GSL0001,35,レギュラー,120,4200
3,2020-12-15,GSL0004,60,灯油,80,4800
4,2020-12-15,CMT0003,2,洗車 C,3000,6000
5,2020-12-18,GSL0004,15,灯油,80,1200
6,2020-12-21,CMT0003,4,洗車 C,3000,12000
7,2020-12-22,GSL0004,22,灯油,80,1760
8,2020-12-22,CMT0005,3,タイヤ交換,4000,12000
9,2020-12-25,GSL0004,32,灯油,80,2560


# ピボットテーブル


In [23]:
df = pd.read_excel('output/all_data_02.xlsx')
df

,Unnamed: 0,企業名,企業コード,請求書No,発行日,発行者,日付,商品コード,品名,数量[L] or [-],単価[¥/L] or [¥],金額
0,0,TTT Inc.,6,12,2020/10/31,岸辺,2020-10-04,GSL0001,レギュラー,20,120,2400
1,1,TTT Inc.,6,12,2020/10/31,岸辺,2020-10-05,GSL0002,ハイオク,45,160,7200
2,2,TTT Inc.,6,12,2020/10/31,岸辺,2020-10-06,GSL0003,軽油,50,100,5000
3,3,TTT Inc.,6,12,2020/10/31,岸辺,2020-10-07,GDS0002,コロン,3,800,2400
4,4,TTT Inc.,6,12,2020/10/31,岸辺,2020-10-08,CMT0004,オイル交換,1,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...
67,67,株式会社ABC,1,3,2020/12/28,桜田,2020-12-20,CMT0004,オイル交換,1,1000,1000
68,68,株式会社ABC,1,3,2020/12/28,桜田,2020-12-22,CMT0005,タイヤ交換,1,4000,4000
69,69,株式会社ABC,1,3,2020/12/28,桜田,2020-12-22,GDS0001,タイヤ,4,8000,32000
70,70,株式会社ABC,1,3,2020/12/28,桜田,2020-12-22,GDS0002,コロン,1,800,800


In [24]:
df.pivot_table(index=['企業名'],
               columns=['品名'],
               values='金額',
               aggfunc=np.sum)

品名,オイル交換,コロン,スナック,タイヤ,タイヤ交換,ハイオク,レギュラー,洗車 A,洗車 B,洗車 C,灯油,軽油
企業名,,,,,,,,,,,,
CDF商事,1000.0,NaN,NaN,NaN,4000.0,7200.0,8400.0,NaN,NaN,6000.0,12400.0,NaN
ICHIGO,1000.0,2400.0,NaN,NaN,NaN,13600.0,2400.0,NaN,1500.0,NaN,NaN,10000.0
STR Corp.,NaN,NaN,1000.0,32000.0,4000.0,7200.0,7200.0,NaN,1500.0,NaN,2400.0,NaN
TTT Inc.,1000.0,2400.0,NaN,NaN,4000.0,19200.0,16800.0,NaN,1500.0,NaN,NaN,44500.0
XYZ株式会社,NaN,NaN,NaN,NaN,NaN,12000.0,6000.0,NaN,NaN,6000.0,6800.0,5000.0
株式会社ABC,1000.0,800.0,300.0,32000.0,4000.0,7200.0,2400.0,500.0,1500.0,3000.0,2400.0,5000.0


In [25]:
df.pivot_table(index=['企業名'],
               values='金額',
               aggfunc=np.sum)

,金額
企業名,
CDF商事,39000
ICHIGO,30900
STR Corp.,55300
TTT Inc.,89400
XYZ株式会社,35800
株式会社ABC,60100


# グラフの作成

In [27]:
# Referenceは参照データ範囲のクラス
import openpyxl
from openpyxl.chart import BarChart, Reference

In [29]:
wb = openpyxl.load_workbook('graphs/企業別当月売上表.xlsx')
sh = wb.active

In [33]:
data = Reference(worksheet=sh, min_col=3, max_col=3, min_row=1, max_row=sh.max_row)
labels = Reference(worksheet=sh, min_col=2, max_col=2, min_row=1, max_row=sh.max_row)

In [38]:
# 棒グラフのインスタンス
chart = BarChart()

# 棒グラフの書式設定
chart.type = 'col'  # 横型の棒グラフ
chart.style = 28  # 文字の大きさ
chart.title = '取引先企業売上'
chart.y_axis.title = '売上'
chart.x_axis.title = '取引先企業名'

# chartにdataを追加する
chart.add_data(data, titles_from_data=True)
chart.set_categories(labels)

# グラフをワークシートに追加する
sh.add_chart(chart, 'E3')

# ブックの保存
wb.save('graphs/企業別当月売上表.xlsx')

In [39]:
# deepcopyはインスタンスを丸ごとコピーするクラス
from copy import deepcopy

In [42]:
# chartのインスタンスをコピーしてchart2にする
chart2 = deepcopy(chart)

# 横型の棒グラフに変更する
chart2.type = 'bar'

sh.add_chart(chart2, 'E10')
wb.save('graphs/企業別当月売上表.xlsx')